# Creating a Sparklines Component for CPU and Memory Usage

This notebook will walk you through:

* Generating fake events
* Creating sparklines with svg elements
* Hooking your "component" up to your CPU and Memory usage

In [ ]:
from vdom import *
from vdom.svg import *
from functools import reduce

Before we work with _real_ data, let's make a function that generates single-valued events. We'll pass it how many events we want and it will return an array of events.

In [ ]:
def generate_events(length=30):
    events = [25] # start our events off at 25
    for x in range(length):
        events.append(generate_next_event(events))
    return events

In [ ]:
def generate_next_event(events):
    # the next value in the event will be ±3 from the previous event,
    # maxing out at 119 and at minimum 0
    return min(max(events[-1] + random.randint(-3, 3), 0), 119)

Let's see how it runs!

In [ ]:
generate_events(5) # Make 5 events

Now it's time to make the view of these events.

Each event will end up as a point on a [`<polyline />`](https://developer.mozilla.org/en-US/docs/Web/SVG/Element/polyline), which is a basic shape for connecting straight lines between many points. The most recent event will show up as a red [`<circle />`](https://developer.mozilla.org/en-US/docs/Web/SVG/Element/circle).

In [ ]:
import random
import time

def minichart(lines, title=""):
    # if lines is a 1D array, we'll assume an equidistant x array
    height = 120
    
    xscale = 30

    # With all the events, we'll scale them according to our height and 
    points = reduce(lambda s, coords: f"{s}{coords[0]*xscale},{height - coords[1]} ", enumerate(lines), "")
    
    last_point = ((len(lines) - 1)*xscale, height - lines[-1])
    
    return svg(
        polyline(fill="none", stroke="black", points=points),
        circle(cx=str(last_point[0]), cy=str(last_point[1]), r="2", fill="red"),
        text(title, x="0", y="20", fontWeight="600", fontFamily="Verdana"),
        width="920", height="120", style={ 'borderTop': '1px solid #f7f7f7', 'paddingTop': '20px'}
    )

We've got our initial setup ready, let's see how that component fares.

In [ ]:
events = generate_events()

handle = display(minichart(events), display_id=True)

Neat. Let's now update it continuously.

In [ ]:
for x in range(150):
    time.sleep(30/1000.)
    events.pop(0)
    events.append(generate_next_event(events))
    handle.update(minichart(events))

## Monitoring system metrics

In [ ]:
# In case we don't have psutil, install it
try:
    import psutil
except ImportError:
    !python -m pip install psutil >/dev/null 2> /dev/null
    import psutil

psutil.cpu_percent()

In [ ]:
psutil.virtual_memory().percent

In [ ]:
cpu_percents = [psutil.cpu_percent()]
vmem_percents = [psutil.virtual_memory().percent]

In [ ]:
def append_cpu_percent(events, max_length=30):
    events.append(psutil.cpu_percent())
    if(len(events) > max_length):
        events.pop(0)
    return events

def append_virtual_memory_percent(events, max_length=30):
    events.append(psutil.virtual_memory().percent)
    if(len(events) > max_length):
        events.pop(0)
    return events

In [ ]:
def system_metrics(cpu_percents, vmem_percents):
    return div(
        minichart(cpu_percents, title=f"CPU {cpu_percents[-1]}%"),
        br(), # lazy padding
        minichart(vmem_percents, title=f"Memory {vmem_percents[-1]}%"),
    )
    

sys_handle = display(system_metrics(cpu_percents, vmem_percents), display_id=True)

for x in range(150):
    time.sleep(100/1000.)
    append_cpu_percent(cpu_percents)
    append_virtual_memory_percent(vmem_percents)
    
    sys_handle.update(system_metrics(cpu_percents, vmem_percents))